# 学習済みのCNNを使用する

## 特徴抽出

モジュールとして提供されている画像分類モデル
* Xception
* Inception V3
* ResNet50
* VGG16
* VGG19
* MobileNet
### VGG16モデルのたたみ込みベースのインスタンス化


In [1]:
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                 include_top=False,
                 input_shape=(256, 256, 3))

Using TensorFlow backend.


In [2]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0     

## データ拡張を行わない高速な特徴抽出
### 学習済みたたみ込みベースを使って特徴量を抽出


In [3]:
import os
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator

mini_metadata = pd.read_csv('mini_metadata.csv',index_col=0)
classes = list(mini_metadata["make_model"].value_counts().index)
classes = sorted(classes)
classes_num = len(mini_metadata.groupby("make_model"))

base_dir = 'mini_pictures'

train_dir = os.path.join(base_dir,'train')
valid_dir = os.path.join(base_dir,'valid')
test_dir = os.path.join(base_dir,'test')

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 32

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 8, 8, 512)) #このshapeになる理由->block5_pool (MaxPooling2D)   (None, 8, 8, 512)
    labels = np.zeros(shape=(sample_count,classes_num)) #labelsのshapeについて確認
    generator = datagen.flow_from_directory(directory,
                                           target_size=(256,256),
                                           batch_size=batch_size,
                                           class_mode='categorical',
                                           classes=classes)
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i*batch_size:(i+1)*batch_size] = features_batch
        labels[i*batch_size:(i+1)*batch_size] = labels_batch
        i += 1
        if i*batch_size >= sample_count:
            # ジェネレータはデータを無限ループで生成するため
            # 画像を一通り処理したらbreak
            break
    return features,labels
train_cnt = 0
valid_cnt = 0
test_cnt = 0
for _class in classes:
    train_label_dir = os.path.join(train_dir, _class)
    train_cnt += len([f for f in os.listdir(train_label_dir) if os.path.isfile(os.path.join(train_label_dir, f))])
    valid_label_dir = os.path.join(valid_dir, _class)
    valid_cnt += len([f for f in os.listdir(valid_label_dir) if os.path.isfile(os.path.join(valid_label_dir, f))])
    test_label_dir = os.path.join(test_dir, _class)
    test_cnt += len([f for f in os.listdir(test_label_dir) if os.path.isfile(os.path.join(test_label_dir, f))])
print(train_cnt,valid_cnt,test_cnt)

train_features, train_labels = extract_features(train_dir,train_cnt)
valid_features, valid_labels = extract_features(valid_dir,valid_cnt)
test_features, test_labels = extract_features(test_dir,test_cnt)

1912 657 663
Found 1912 images belonging to 20 classes.
Found 657 images belonging to 20 classes.
Found 663 images belonging to 20 classes.


In [6]:
# train_features = np.reshape(train_features, (train_cnt, 8 * 8 * 512))
# valid_features = np.reshape(valid_features, (valid_cnt, 8 * 8 * 512))
# test_features = np.reshape(test_features, (test_cnt, 8 * 8 * 512))

In [5]:
from keras import models 
from keras import layers 
from keras import optimizers

model = models.Sequential()
model.add(layers.Conv2D(256,(3, 3), activation='relu', input_shape=(8, 8, 512)))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(classes_num, activation='softmax'))

model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
             loss='categorical_crossentropy',
             metrics=['acc'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 6, 6, 256)         1179904   
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2359552   
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                5140      
Total params: 3,544,596
Trainable params: 3,544,596
Non-trainable params: 0
_________________________________________________________________


In [6]:
history = model.fit(train_features, train_labels,
                   epochs=100,
                   batch_size=32,
                   validation_data=(valid_features, valid_labels))

Train on 1912 samples, validate on 657 samples
Epoch 1/100
1912/1912 [==============================] - 9s 5ms/step - loss: 2.9651 - acc: 0.1004 - val_loss: 2.7490 - val_acc: 0.2359
Epoch 2/100
1912/1912 [==============================] - 8s 4ms/step - loss: 2.6830 - acc: 0.2103 - val_loss: 2.5455 - val_acc: 0.3257
Epoch 3/100
1912/1912 [==============================] - 8s 4ms/step - loss: 2.4324 - acc: 0.3086 - val_loss: 2.3061 - val_acc: 0.4110
Epoch 4/100
1912/1912 [==============================] - 8s 4ms/step - loss: 2.2152 - acc: 0.3776 - val_loss: 2.0605 - val_acc: 0.5190
Epoch 5/100
1912/1912 [==============================] - 9s 5ms/step - loss: 1.9875 - acc: 0.4430 - val_loss: 1.8648 - val_acc: 0.6164
Epoch 6/100
1912/1912 [==============================] - 9s 5ms/step - loss: 1.7805 - acc: 0.5199 - val_loss: 1.6837 - val_acc: 0.6393
Epoch 7/100
1912/1912 [==============================] - 9s 5ms/step - loss: 1.6331 - acc: 0.5513 - val_loss: 1.5331 - val_acc: 0.6454
Epoch 8/

Epoch 61/100
1912/1912 [==============================] - 8s 4ms/step - loss: 0.0410 - acc: 0.9969 - val_loss: 0.1052 - val_acc: 0.9726
Epoch 62/100
1912/1912 [==============================] - 8s 4ms/step - loss: 0.0432 - acc: 0.9963 - val_loss: 0.0977 - val_acc: 0.9802
Epoch 63/100
1912/1912 [==============================] - 9s 5ms/step - loss: 0.0402 - acc: 0.9969 - val_loss: 0.1061 - val_acc: 0.9817
Epoch 64/100
1912/1912 [==============================] - 9s 5ms/step - loss: 0.0385 - acc: 0.9974 - val_loss: 0.0990 - val_acc: 0.9772
Epoch 65/100
1912/1912 [==============================] - 10s 5ms/step - loss: 0.0342 - acc: 0.9974 - val_loss: 0.0958 - val_acc: 0.9772
Epoch 66/100
1912/1912 [==============================] - 9s 5ms/step - loss: 0.0322 - acc: 0.9974 - val_loss: 0.0996 - val_acc: 0.9833
Epoch 67/100
1912/1912 [==============================] - 9s 5ms/step - loss: 0.0323 - acc: 0.9979 - val_loss: 0.0954 - val_acc: 0.9802
Epoch 68/100
1912/1912 [=======================

In [7]:
import matplotlib.pyplot as plt
%matplotlib inline

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

#正解率をプロット
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

#損失値をプロット
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validatin loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [8]:
model.save("VGG16_model_mini_2.h5")